# GI and QAOA try 2

In [1]:
import numpy as np

In [2]:
def create_qubo(E1,E2,vertices,p):
    Q = np.zeros((vertices*vertices, vertices*vertices))
    
    # Constraint 1: penalty if several mappings from same source
    for i in range(vertices): 
        for j in range(vertices): 
            for k in range(j+1,vertices): 
                Q[i*vertices+j,i*vertices+k]=p 

    # Constaint 2: penalty if several mappings to same target
    for i in range(vertices): 
        for j in range(vertices): 
            for k in range(j+1,vertices): 
                Q[i+vertices*j,i+vertices*k]=p 
                
    # Constraint 3: -1 for each succesfully mapped edge: (x1,y1) -> (x2,y2) 
    #    two possible mappings: (x1->x2, y1->y2) or (x1->y2,y1->x2)
    for e1 in E1: 
        for e2 in E2: 
            Q[e1[0]*vertices+e2[0], e1[1]*vertices+e2[1]] -= 1
            Q[e1[0]*vertices+e2[1], e1[1]*vertices+e2[0]] -= 1
            
    # All quadratic coefficients in lower triangle to upper triangle
    for i in range(vertices*vertices): 
        for j in range(i):
            Q[j,i] += Q[i,j]
            Q[i,j] = 0
    return Q

In [3]:
def result_info(sampleset, e, labels):
    le = int(sampleset[0].fval)
    print('Lowest energy should be:',-e)
    print('Lowest energy was:',le)
    results = []
    if -e!=le:
        print('Graphs are NOT isomorphic')
    else:
        print('Graphs are isomorphic')
        for sample in sampleset:
            if sample.fval==le:
                m = ''
                for i,x in enumerate(sample.x):
                    if x==1:
                        m += str(labels[i])+', '
                print('Mapping: '+m)

In [21]:
#vertices = 3
#E1 = np.array([(0, 1), (1, 2)])
#E2 = np.array([(0, 1), (0, 2)]) 

vertices = 2
E1 = np.array([(0, 1)])
E2 = np.array([(1, 0)]) 

In [22]:
labels = {}
for i in range(vertices):
    for j in range(vertices):
        labels[i*vertices+j] = (i,j)
            
p = len(E1)
print('Penalty:',p)

Penalty: 1


In [23]:
Q = create_qubo(E1,E2,vertices,p)
qubosize= vertices*vertices

## Build hamiltonian $H_p$

In [24]:
from qiskit.quantum_info import SparsePauliOp, Pauli
oplist=[]
offset = 0
for i in range(qubosize):
    if Q[i,i]!=0:
        oplist.append(('Z',i,-Q[i,i]/2))
    else:
        offset += 1/2
for i in range(qubosize):
    for j in range(i+1,qubosize):
        if Q[i,j]!=0:
            oplist.append(('ZZ',[i,j],Q[i,j]/4))
            oplist.append(('Z',[i],-Q[i,j]/4))
            oplist.append(('Z',[j],-Q[i,j]/4))
        else:
            offset += 1/4
                          
H_p = SparsePauliOp.from_sparse_list(oplist, num_qubits=qubosize).simplify()

In [26]:
print(H_p)
print(offset)
print(qubosize)


SparsePauliOp(['IIZZ', 'IIIZ', 'IIZI', 'IZIZ', 'IZII', 'ZIIZ', 'ZIII', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[ 0.25+0.j, -0.25+0.j, -0.25+0.j,  0.25+0.j, -0.25+0.j, -0.25+0.j,
 -0.25+0.j, -0.25+0.j,  0.25+0.j,  0.25+0.j])
2.0
4


## Build QAOA 

In [10]:
from qiskit import BasicAer, transpile
from qiskit.circuit.library import QAOAAnsatz

#ansatz = QAOAAnsatz(H_p, reps=2) 

In [12]:
from qiskit.primitives import Sampler
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA

backend = BasicAer.get_backend('qasm_simulator')
sampler = Sampler()
qaoa = QAOA(sampler, COBYLA(), reps=2, initial_point=[0.0, 0.0, 0.0, 0.0])
result = qaoa.compute_minimum_eigenvalue(H_p) # This includes automatically mixer operator

In [13]:
result.best_measurement

{'state': 6,
 'bitstring': '0110',
 'value': (-1.5+0j),
 'probability': 0.4122789886803482}

In [14]:
print('Binary variable vector:',result.best_measurement['bitstring'])
print('Energy level:',np.real(result.best_measurement['value']+offset))

Binary variable vector: 0110
Energy level: 0.5


In [15]:
print('Optimal parameters:',result.optimal_point)
print('Optimal value:',result.optimal_value)
print('Number of optimizer evaluations:',result.optimizer_evals)
print('Optimizer result:',result.optimizer_result)
print('Time (s):',result.optimizer_time)
print('Eigenvalue:',result.eigenvalue)
print('Number of cost optimizer evaluations:',result.cost_function_evals)

Optimal parameters: [-0.32616107  0.32721212  1.15000964 -1.95711985]
Optimal value: -1.298474275763471
Number of optimizer evaluations: None
Optimizer result: {   'fun': -1.298474275763471,
    'jac': None,
    'nfev': 802,
    'nit': None,
    'njev': None,
    'x': array([-0.32616107,  0.32721212,  1.15000964, -1.95711985])}
Time (s): 12.70673394203186
Eigenvalue: -1.298474275763471
Number of cost optimizer evaluations: 802


In [20]:
result.optimal_circuit.decompose(reps=2).draw()

┌─────────────┐               ┌───────────────┐               »
   q_0: ┤ U3(π/2,0,π) ├─■─────────────┤ Rz(-0.5*γ[0]) ├─■─────────────»
        ├─────────────┤ │ZZ(0.5*γ[0]) ├───────────────┤ │             »
   q_1: ┤ U3(π/2,0,π) ├─■─────────────┤ Rz(-0.5*γ[0]) ├─┼─────────────»
        ├─────────────┤               └───────────────┘ │ZZ(0.5*γ[0]) »
   q_2: ┤ U3(π/2,0,π) ├─────────────────────────────────■─────────────»
        ├─────────────┤                                               »
   q_3: ┤ U3(π/2,0,π) ├───────────────────────────────────────────────»
        └─────────────┘                                               »
meas: 4/══════════════════════════════════════════════════════════════»
                                                                      »
«                                          ┌──────────────┐               »
«   q_0: ──────────────────■───────────────┤ Rx(2.0*β[0]) ├───────────────»
«                          │               └──────────────┘               »
«   q_1: ──────────────────┼────────────────■───────────────■─────────────»
«        ┌───────────────┐ │                │ZZ(-0.5*γ[0])  │             »
«   q_2: ┤ Rz(-0.5*γ[0]) ├─┼────────────────■───────────────┼─────────────»
«        └───────────────┘ │ZZ(-0.5*γ[0]) ┌───────────────┐ │ZZ(0.5*γ[0]) »
«   q_3: ──────────────────■──────────────┤ Rz(-0.5*γ[0]) ├─■─────────────»
«                                         └───────────────┘               »
«meas: 4/═════════════════════════════════════════════════════════════════»
«                                                                         »
«                                        ┌───────────────┐               »
«   q_0: ─────────────────■──────────────┤ Rz(-0.5*γ[1]) ├─■─────────────»
«        ┌──────────────┐ │ZZ(0.5*γ[1])  ├───────────────┤ │             »
«   q_1: ┤ Rx(2.0*β[0]) ├─■──────────────┤ Rz(-0.5*γ[1]) ├─┼─────────────»
«        └──────────────┘┌──────────────┐└───────────────┘ │ZZ(0.5*γ[1]) »
«   q_2: ─■──────────────┤ Rx(2.0*β[0]) ├──────────────────■─────────────»
«         │ZZ(0.5*γ[0])  ├──────────────┤                                »
«   q_3: ─■──────────────┤ Rx(2.0*β[0]) ├────────────────────────────────»
«                        └──────────────┘                                »
«meas: 4/════════════════════════════════════════════════════════════════»
«                                                                        »
«                                          ┌──────────────┐               »
«   q_0: ──────────────────■───────────────┤ Rx(2.0*β[1]) ├───────────────»
«                          │               └──────────────┘               »
«   q_1: ──────────────────┼────────────────■───────────────■─────────────»
«        ┌───────────────┐ │                │ZZ(-0.5*γ[1])  │             »
«   q_2: ┤ Rz(-0.5*γ[1]) ├─┼────────────────■───────────────┼─────────────»
«        └───────────────┘ │ZZ(-0.5*γ[1]) ┌───────────────┐ │ZZ(0.5*γ[1]) »
«   q_3: ──────────────────■──────────────┤ Rz(-0.5*γ[1]) ├─■─────────────»
«                                         └───────────────┘               »
«meas: 4/═════════════════════════════════════════════════════════════════»
«                                                                         »
«                                         ░ ┌─┐         
«   q_0: ─────────────────────────────────░─┤M├─────────
«        ┌──────────────┐                 ░ └╥┘┌─┐      
«   q_1: ┤ Rx(2.0*β[1]) ├─────────────────░──╫─┤M├──────
«        └──────────────┘┌──────────────┐ ░  ║ └╥┘┌─┐   
«   q_2: ─■──────────────┤ Rx(2.0*β[1]) ├─░──╫──╫─┤M├───
«         │ZZ(0.5*γ[1])  ├──────────────┤ ░  ║  ║ └╥┘┌─┐
«   q_3: ─■──────────────┤ Rx(2.0*β[1]) ├─░──╫──╫──╫─┤M├
«                        └──────────────┘ ░  ║  ║  ║ └╥┘
«meas: 4/════════════════════════════════════╩══╩══╩══╩═
«                                            0  1  2  3

In [106]:
job = sampler.run(qc, parameter_values=[0.0,0.0, 0.0,0.0])

In [107]:
i=0
e=0.0
for k,v in job.result().quasi_dists[0].items():
    if v>e:
        e=v
        i=k
print(i,e)

0 0.001953125


In [108]:
def bitfield(n, L):
    result = np.binary_repr(n, L)
    return [int(digit) for digit in result]
x = bitfield(i, 9)
x.reverse()
x

[0, 0, 0, 0, 0, 0, 0, 0, 0]